# 美股 API 測試

In [1]:
import pandas as pd
import requests
import json
import configparser
from sqlalchemy import create_engine

In [2]:
# 資料庫連接設定

# 讀取 config.ini 檔案
config = configparser.ConfigParser()
config.read('../../DB_Processor/config.ini')


# 從 config.ini 中獲取資料庫連接設定
username = config['database']['user']
password = config['database']['password']
database = config['database']['db']
host = config['database']['host']
port = config['database']['port']

engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database}')
# # data = pd.read_csv(csv_path)

---

In [3]:
# query = "SELECT id, company_symbol FROM company WHERE country!='TW'"
# company_mapping = pd.read_sql_query(query, engine)
# company_mapping.set_index('company_symbol', inplace=True)

In [4]:
# company_mapping

In [5]:
# company_list = company_mapping.index.tolist()

In [6]:
# comp_test = 'AAPL'
# BASE_URL = 'https://financialmodelingprep.com/api/v3/'
# STATEMENT_TYPE = {
    
#     'income-statement/': ['date', 'symbol', 'period', 'incomeBeforeTax', 'netIncome', 'eps', 'ebitda', 'revenue'], 

#     'balance-sheet-statement/':['date', 'symbol', 'period', 'longTermInvestments', 'propertyPlantEquipmentNet', 'netDebt', 'totalLiabilitiesAndStockholdersEquity','totalStockholdersEquity', 'retainedEarnings'],

#     'cash-flow-statement/':['date', 'symbol', 'period', 'freeCashFlow', 'netCashProvidedByOperatingActivities'],


#     'ratio/': ['date', 'symbol', 'period', 'returnOnEquity', 'dividendPayoutRatio', 'priceEarningsRatio'],

#     }

# PERIOD = ['quarter', 'annual']
# LIMIT = 0 # 限制筆數，可以加在URL最後方，顯示最新的N筆資料，0代表沒有限制

# API_KEY = config['api']['financialmodelingprep_api_key']
# START_DATE = '2000-01-01'
# END_DATE = '2024-09-30'

In [7]:
# # url = f'{BASE_URL}{comp_test}?from={START_DATE}&apikey={API_KEY}' # 股價
# url = f'{BASE_URL}{STATEMENT_TYPE}{comp_test}?period={PERIOD}&apikey={API_KEY}&limit={LIMIT}' # 財報


# response = requests.get(url)
# data = response.json()

In [8]:
# data

In [9]:
# all_data = pd.DataFrame()

# for company in company_list[:2]:
#     url = f'{BASE_URL}{company}?from={START_DATE}&to={END_DATE}&apikey={API_KEY}'
#     response = requests.get(url)
#     data = response.json()

#     # print(data)
#     # Extracting symbol and historical data
#     symbol = data['symbol']
#     historical_data = data['historical']

#     # Create a DataFrame and add the symbol column
#     df = pd.DataFrame(historical_data)
#     df['symbol'] = symbol

#     # Selecting the required columns
#     df = df[['date', 'open', 'high', 'low', 'adjClose', 'volume', 'symbol']]

#     # Concatenating the data
#     all_data = pd.concat([all_data, df], ignore_index=True)


In [10]:
# df_merged = all_data.merge(company_mapping, how='left', left_on='symbol', right_index=True)
# # 移除不需要的 company_symbol 欄位
# df_merged = df_merged.drop(columns=['symbol'])

In [11]:
# df_merged['date'] = pd.to_datetime(df_merged['date'])

# data_to_insert = df_merged.rename(columns={'id': 'company_id', 'adjClose': 'close'})

In [12]:
# data_to_insert

---

In [13]:
# df = pd.DataFrame(data)

In [14]:
# all_data = pd.DataFrame()

# for company in company_list[:2]:
#     # url = f'{BASE_URL}{company}?from={START_DATE}&to={END_DATE}&apikey={API_KEY}'
#     # url = f'{BASE_URL}{STATEMENT_TYPE}{comp_test}?period={PERIOD}&apikey={API_KEY}&limit={LIMIT}'
#     url = f'{BASE_URL}ratios/{company}?period={PERIOD}&apikey={API_KEY}'

#     response = requests.get(url)
#     data = response.json()

#     # symbol = data['symbol']

#     # Create a DataFrame and add the symbol column
#     df = pd.DataFrame(data)

#     # Selecting the required columns
#     df = df[['date', 'symbol', 'period', 'returnOnEquity', 'dividendPayoutRatio', 'priceEarningsRatio']]

#     # Concatenating the data
#     all_data = pd.concat([all_data, df], ignore_index=True)


In [15]:
# all_data

In [16]:


BASE_URL = 'https://financialmodelingprep.com/api/v3/'
STATEMENT_TYPE = {
    'income-statement/': ['date', 'symbol', 'period', 'incomeBeforeTax', 'netIncome', 'eps', 'ebitda', 'revenue'],
    'balance-sheet-statement/': ['date', 'symbol', 'period', 'longTermInvestments', 'propertyPlantEquipmentNet', 'netDebt', 'totalLiabilitiesAndStockholdersEquity', 'totalStockholdersEquity', 'retainedEarnings'],
    'cash-flow-statement/': ['date', 'symbol', 'period', 'freeCashFlow', 'netCashProvidedByOperatingActivities'],
    'ratios/': ['date', 'symbol', 'period', 'returnOnEquity', 'dividendPayoutRatio', 'priceEarningsRatio'],
}
PERIODS = ['quarter', 'annual']
LIMIT = 0  # 限制筆數，0代表沒有限制
API_KEY = config['api']['financialmodelingprep_api_key']  # 請替換為您的 API 密鑰

def fetch_data(url):
    """從 API 獲取數據"""
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"API request failed with status code {response.status_code}")

def process_statement(statement_data, indicators):
    """
    處理單個財務報表的數據
    """
    rows = []
    for item in statement_data:
        for indicator in indicators[3:]:  # 跳過 'date', 'symbol', 'period'
            rows.append({
                'symbol': item['symbol'],
                'date': item['date'],
                'period': item['period'],
                'indicator_name': f"{indicator}",
                'indicator_value': item.get(indicator, None)
            })
    return pd.DataFrame(rows)

def fetch_and_process_data(symbols):
    """獲取並處理所有公司的所有財務數據"""
    all_data = []
    
    for symbol in symbols:
        for statement, indicators in STATEMENT_TYPE.items():
            for period in PERIODS:
                url = f"{BASE_URL}{statement}{symbol}?period={period}&apikey={API_KEY}&limit={LIMIT}"
                try:
                    data = fetch_data(url)
                    df = process_statement(data, indicators)
                    all_data.append(df)
                except Exception as e:
                    print(f"Error processing {symbol} {statement} {period}: {str(e)}")
    
    return pd.concat(all_data, ignore_index=True)

def main():
    # 要獲取數據的公司代碼列表
    symbols = ['AAPL', 'GOOGL', 'MSFT']
    
    # 獲取並處理所有數據
    result_df = fetch_and_process_data(symbols)
    
    # 保存為 CSV 文件
    result_df.to_csv('../data/US_data/US_financial_data_test.csv', index=False)

if __name__ == "__main__":
    main()

Data has been saved to financial_data.csv


In [17]:
df = pd.read_csv('../data/US_data/US_financial_data_test.csv')
df

,symbol,date,period,indicator_name,indicator_value
0,AAPL,2024-06-29,Q3,incomeBeforeTax,2.549400e+10
1,AAPL,2024-06-29,Q3,netIncome,2.144800e+10
2,AAPL,2024-06-29,Q3,eps,1.400000e+00
3,AAPL,2024-06-29,Q3,ebitda,2.820200e+10
4,AAPL,2024-06-29,Q3,revenue,8.577700e+10
...,...,...,...,...,...
7785,MSFT,1987-06-30,FY,dividendPayoutRatio,NaN
7786,MSFT,1987-06-30,FY,priceEarningsRatio,3.541700e+01
7787,MSFT,1986-06-30,FY,returnOnEquity,2.821249e-01
7788,MSFT,1986-06-30,FY,dividendPayoutRatio,NaN


In [18]:
# query = "SELECT id, company_symbol FROM company WHERE country!='TW'"
# company_mapping = pd.read_sql_query(query, engine)
# company_mapping.set_index('company_symbol', inplace=True)
# company_list = company_mapping.index.tolist()


# query = "SELECT id, us_indicator FROM singleindicator"
# indicators = pd.read_sql_query(query, engine)



# # 將company_id加入df當中、並將indicator_name轉換成對應的id
# df_merged = df.merge(company_mapping, how='left', left_on='symbol', right_index=True)
# indicator_mapping = indicators.reset_index().set_index('us_indicator')
# df_merged = df_merged.merge(indicator_mapping, how='left', left_on='indicator_name', right_index=True)

# df_merged = df_merged.rename(columns={'id_x': 'company_id', 'id_y': 'indicator_id'})

In [19]:
# 處理 company_id
company_ids = pd.read_sql('SELECT id, company_symbol FROM company WHERE country != \'TW\'', con=engine)
company_id_map = dict(zip(company_ids['company_symbol'], company_ids['id']))
df['company_id'] = df['symbol'].map(company_id_map)

# 處理 indicator_id
indicator_ids = pd.read_sql('SELECT id, us_indicator FROM singleindicator', con=engine)
indicator_id_map = dict(zip(indicator_ids['us_indicator'], indicator_ids['id']))
df['indicator_id'] = df['indicator_name'].map(indicator_id_map)

# 處理 period_id
period_id_map = {'Q1': 1, 'Q2': 2, 'Q3': 3, 'Q4': 4, 'FY': 5}
df['period_id'] = df['period'].map(period_id_map)


# 處理股利支付率的空值
query = "SELECT id FROM singleindicator WHERE us_indicator = 'dividendPayoutRatio'"
dividend_payout_id = pd.read_sql(query, con=engine)['id'].values[0]

df.loc[df['indicator_id'] == dividend_payout_id, 'indicator_value'] = df.loc[df['indicator_id'] == dividend_payout_id, 'indicator_value'].fillna(0)

In [20]:
df['date'] = pd.to_datetime(df['date'])

In [21]:
col_to_insert = ['company_id', 'date', 'period_id', 'indicator_id', 'indicator_value']
data_to_insert = df[col_to_insert]
data_to_insert = data_to_insert[data_to_insert['date']>='2000-01-01']

In [22]:
data_to_insert[data_to_insert['indicator_id']==14]

,company_id,date,period_id,indicator_id,indicator_value
2423,1935,2024-06-29,3,14,37.530978
2426,1935,2024-03-30,2,14,27.942505
2429,1935,2023-12-30,1,14,22.010958
2432,1935,2023-09-30,4,14,29.085850
2435,1935,2023-07-01,3,14,38.288645
...,...,...,...,...,...
7735,3503,2004-06-30,5,14,37.773467
7738,3503,2003-06-30,5,14,36.507465
7741,3503,2002-06-30,5,14,36.358875
7744,3503,2001-06-30,5,14,53.075551
